In [ ]:
from pprint import pprint

import cv2
import matplotlib.pyplot as plt
import numpy as np
from requests import post

from box_utils import predict

In [ ]:
prediction_url = 'EDIT'

In [ ]:
def get_model_response(image_array):
    payload = {
        'inputs': [
            {
                'name': 'input',
                'shape': [1, 3, 480, 640],
                'datatype': 'FP32',
                'data': image_array.tolist(),
            }
        ]
    }
    raw_response = post(prediction_url, json=payload)
    response = raw_response.json()
    model_outputs = response['outputs']
    for output in model_outputs:
        if output['name'] == 'scores':
            scores = np.array(output['data'])
        elif output['name'] == 'boxes':
            boxes = np.array(output['data'])
    
    return scores.reshape(1, int(len(scores)/2), 2), boxes.reshape(1, int(len(boxes)/4), 4)

def scale(box):
    width = box[2] - box[0]
    height = box[3] - box[1]
    maximum = max(width, height)
    dx = int((maximum - width)/2)
    dy = int((maximum - height)/2)

    bboxes = [box[0] - dx, box[1] - dy, box[2] + dx, box[3] + dy]
    return bboxes

# crop image
def cropImage(image, box):
    num = image[box[1]:box[3], box[0]:box[2]]
    return num

# face detection method
def faceDetector(orig_image, threshold = 0.7):
    image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(orig_image, (640, 480))
    image_mean = np.array([127, 127, 127])
    image = (image - image_mean) / 128
    image = np.transpose(image, [2, 0, 1])
    image = np.expand_dims(image, axis=0)
    image = image.astype(np.float32)

    confidences, boxes = get_model_response(image)
    boxes, labels, probs = predict(orig_image.shape[1], orig_image.shape[0], confidences, boxes, threshold)
    return boxes, labels, probs

In [ ]:
img_path = '1.jpg'
color = (255, 128, 0)

orig_image = cv2.imread(img_path)
image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)
boxes, labels, probs = faceDetector(orig_image)

for i in range(boxes.shape[0]):
    box = scale(boxes[i, :])
    cv2.rectangle(image, (box[0], box[1]), (box[2], box[3]), color, 4)
    plt.imshow(image)
